In [ ]:
import importlib, tools
importlib.reload(tools)
from tools import plot_surface, read_cloud, create_surface_mesh, plot_points, save_cloud

%matplotlib qt
# %matplotlib inline

In [ ]:
import numpy as np


def ransac_fit_plane(
    voxels: np.ndarray, inlier_thresh=0.1, iterations=100
):  # voxels : x,y,z
    """Fits a plane to points using the RANSAC algorithm

    Args:
        voxels (np.ndarray): points to fit the plane
        inlier_thresh (int, optional): distance from the plane for filtering the inliers. Defaults to 1.
        iterations (int, optional): number of iterations. Defaults to 100.

    Returns:
        np.ndarray: array of coefficients for A*x+B*y+C*z+D=0 equation
        int: number of inliers for the returned plane

    """
    out_inliers = []
    out_fit = np.zeros(4)
    for _ in range(iterations):
        pts = voxels[np.random.choice(voxels.shape[0], 3, replace=False)]
        v_a = pts[0] - pts[2]
        v_b = pts[1] - pts[2]
        norm_a = v_a / np.linalg.norm(v_a)
        norm_b = v_b / np.linalg.norm(v_b)
        v_plane = np.cross(norm_a, norm_b)  # A,B,C coefficients
        v_plane = v_plane / np.linalg.norm(v_plane)  # normalize plane vector
        D_plane = -np.sum(np.multiply(v_plane, pts[2]))
        distances = calc_plane_distance(voxels, np.array([*v_plane, D_plane]))
        inliers = voxels[distances < inlier_thresh]
        if inliers.shape[0] > len(out_inliers):
            out_inliers = inliers
            out_fit = [*v_plane, D_plane]
    return np.array(out_fit), out_inliers


def calc_plane_distance(voxels: np.ndarray, fit: np.ndarray):
    """Calculates distances of points from the plane

    Args:
        voxels (np.ndarray): N x 3 array of points (x,y,z coordinates)
        fit (np.ndarray): 4 element array of plane coefficients
    """
    return np.abs((np.matmul(voxels, fit[:3]) + fit[3]) / np.linalg.norm(fit[:3]))

In [ ]:
# fname = "z_plane"
# xyz = read_cloud(f"../clouds/{fname}.xyz")

# fit, _ = ransac_fit_plane(xyz)

# n_points = 1000
# x = np.linspace(xyz[:, 0].min(), xyz[:, 0].max(), n_points)
# y = np.linspace(xyz[:, 1].min(), xyz[:, 1].max(), n_points)
# xy1 = np.column_stack([x, y, np.ones(shape=(n_points))])
# z = np.matmul(xy1, fit[[0,1,3]]/fit[2])

# def z_calc_mesh(x, y):
#     return fit[0]/fit[2] * x + fit[1]/fit[2] * y + fit[3]/fit[2]

# # save_cloud(f"../clouds/{fname}_fit.xyz", zip(x,y,z))

# ax, fig = plot_points(voxels=xyz)
# ax, fig = plot_surface(
#     *create_surface_mesh(
#         (xyz[:, 0].min(), xyz[:, 0].max()),
#         (xyz[:, 1].min(), xyz[:, 1].max()),
#         z_calc_mesh,
#         10,
#     ),
#     ax=ax,
#     alpha=0.2,
#     color="y",
# )
# lg = fig.legend(["Chmura punktów", "Dopasowana płaszczyna"])

In [ ]:
from sklearn.cluster import KMeans


def run_fit_planes(conf):
    multicloud = np.concatenate(
        [
            read_cloud(f"../clouds/{fname}.xyz") + translate_vector
            for fname, translate_vector in conf
        ]
    )
    ax, fig = plot_points(voxels=multicloud)
    # ax.set_title("Test cloud built from primitives")
    # fig.tight_layout()

    clusterer = KMeans(n_clusters=len(conf))

    clusters = clusterer.fit_predict(multicloud)
    clouds = {idx: multicloud[clusters == idx] for idx in np.unique(clusters)}
    ax = None
    for idx, cloud in clouds.items():
        ax, fig = plot_points(voxels=cloud, ax=ax, label=f"cluster {idx}")

        fit, inliers = ransac_fit_plane(cloud, inlier_thresh=1)

        def z_calc_mesh(x, y):
            return -(fit[0] / fit[2] * x + fit[1] / fit[2] * y + fit[3] / fit[2])

        def classify_fit():
            if inliers.shape[0] < 0.7 * cloud.shape[0]:
                return "not a plane"
            elif abs(fit[2]) > 0.8:
                return "a horizontal plane"
            return "a vertical plane"

        print(
            "Plane fitted for cluster {}: {:.2f}*x + ({:.2f}*y) + ({:.2f}*z) + ({:.2f}) = 0;\tCluster is {}".format(
                idx, *fit, classify_fit()
            )
        )

        ax, fig = plot_surface(
            *create_surface_mesh(
                (cloud[:, 0].min(), cloud[:, 0].max()),
                (cloud[:, 1].min(), cloud[:, 1].max()),
                z_calc_mesh,
                10,
            ),
            ax=ax,
            alpha=0.4,
            color=ax.collections[-1].get_facecolor(),
            label=f"plane {idx}",
        )

    for i, f in enumerate([ax.set_xlim3d, ax.set_ylim3d, ax.set_zlim3d]):
        f(np.min(multicloud[:, i]), np.max(multicloud[:, i]))
    ax.set_aspect("equal", "box")
    fig.legend()

In [ ]:
conf = [
    ["z_plane", np.array([0, 0, -80])],
    ["x_plane", np.array([-60, 0, 20])],
    ["cylinder", np.array([0, 60, 20])],
]

run_fit_planes(conf)

In [ ]:
conf = [
    ["z_plane", np.array([0, 0, 0])],
    ["x_plane", np.array([-40, 0, 0])],
    ["cylinder", np.array([40, 0, 0])],
]

run_fit_planes(conf)

In [ ]:
from pyransac3d import Plane

room_full = read_cloud("../clouds/conferenceRoom_1.txt", delimiter=" ")

iter = 6
ax = None
for i in range(iter):
    equation, inliers = Plane().fit(room_full[:, 0:3], thresh=0.05, maxIteration=200)
    print(
        "Equation of plane {} with {} inliers: {:.2f}*x + ({:.2f}*y) + ({:.2f}*z) + ({:.2f}) = 0".format(
            i, inliers.shape[0], *equation
        )
    )
    # ax,fig = plot_points(voxels=room_full[inliers,0:3], label=f'plane_{i}', ax=ax)
    save_cloud(f"../clouds/room_clouds/plane_{i}.xyz", room_full[inliers])
    room_full = np.delete(room_full, inliers, axis=0)
# lg = ax.legend()